# Imports

In [ ]:
from fastai.vision.all import *
import shutil

In [ ]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything(27)

# Data 

In [ ]:
healthy_plants_path = '/kaggle/input/healthy-and-wilted-houseplant-images/houseplant_images/healthy'
wilted_plants_path = '/kaggle/input/healthy-and-wilted-houseplant-images/houseplant_images/wilted'

healthy = get_image_files(healthy_plants_path)
wilted = get_image_files(wilted_plants_path)

In [ ]:
im = Image.open(healthy[1])
im.to_thumb(224,224)

# Datablock and DataLoader

In [ ]:
def parent_label(fileName):
    if ("/wilted" in str(fileName)):
        return "wilted"
    else:
        return "healthy"

In [ ]:
path = Path('/kaggle/input/healthy-and-wilted-houseplant-images')
dls = DataBlock (
    blocks = (ImageBlock, CategoryBlock), 
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label, 
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

In [ ]:
dls.show_batch()

# Model Creation

In [ ]:
%%capture

learn = None

learn = vision_learner(dls, resnet18, metrics=[accuracy, F1Score()], cbs=MixUp(alpha=1.))

In [ ]:
dls.vocab

# Model Training

In [ ]:
learn.fine_tune(7)

# Model Validation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
learn.export('houseplantmodel.pkl')